In [5]:
# load packages
import pandas as pd
import numpy as np

**Note: This notebook makes assumptions about where the MultiPic (vers. 1) and the SUBTLEX-DE corpora are saved in relation to the location of the notebook.**

The paths from where they are loaded may need to be adapted for you to be able to run the notebook -- see the next cell.

If you haven't downloaded the databases yet, you can do so here:
- MultiPic (version 1): https://www.bcbl.eu/databases/multipic/ 
- SUBTLEX-DE: http://crr.ugent.be/SUBTLEX-DE/

In [6]:
# PATHS
multipic_path = '../../external_resources/multipic/German_MultiPic_version_1.csv'
subtlex_path = '../../external_resources/frequencies/SUBTLEX-DE_cleaned_with_Google00.txt'

In [7]:
# load MultiPic database as dataframe
multipic_df = pd.read_csv(multipic_path,sep=';', decimal=',') 
# drop redundant column
multipic_df.drop(columns=['PICTURE'], inplace=True)

# check out dataframe
multipic_df

,ITEM,NAME1,NAME2,H_INDEX,PERCENTAGE_MODAL_NAME,PERCENTAGE_VALID_RESPONSES,NUMBER_DIFFERENT_RESPONSES,PERCENTAGE_UNKNOWN_RESPONSES,PERCENTAGE_IDIOSYNCRATIC_RESPONSES,VISUAL_COMPLEXITY
0,1,maus,NaN,0.000000,100.000000,97,1,0,3,3.02
1,2,reifen,NaN,1.163235,69.696970,99,3,0,1,2.67
2,3,treppenstufe,NaN,1.426663,46.464646,99,3,0,1,1.89
3,4,altar,NaN,0.968778,77.319588,97,4,1,2,3.40
4,5,schulter,NaN,0.862355,82.978723,94,4,1,5,2.01
...,...,...,...,...,...,...,...,...,...,...
745,746,kegel,NaN,0.516599,90.721649,97,3,1,2,1.69
746,747,badewanne,NaN,0.468996,90.000000,100,2,0,0,2.57
747,748,muschel,NaN,1.102087,73.195876,97,3,1,2,3.25
748,749,muelleimer,NaN,1.678743,50.000000,98,5,0,2,3.32


In [8]:
# load cleaned SUBTLEX-DE dataset as dataframe
subtlex_df = pd.read_csv(subtlex_path, sep='\t', decimal=',', encoding='latin_1',)
# drop redundant column
subtlex_df.drop(columns=['Unnamed: 10'], inplace=True)

# check out dataframe
subtlex_df

,Word,WFfreqcount,spell-check OK (1/0),CUMfreqcount,SUBTLEX,lgSUBTLEX,Google00,Google00cum,Google00pm,lgGoogle00
0,Genaui,1,0,1,0.04,0.301,1,1,0.00017,0.301
1,hinweggewandt,1,1,1,0.04,0.301,1,1,0.00017,0.301
2,Eskimohunden,1,1,1,0.04,0.301,1,1,0.00017,0.301
3,nieam,1,0,1,0.04,0.301,1,1,0.00017,0.301
4,Riccordi,1,0,1,0.04,0.301,1,1,0.00017,0.301
...,...,...,...,...,...,...,...,...,...,...
190495,von,109563,1,116691,4594.31,5.067,61655024,63352844,10470.92742,7.802
190496,in,182149,1,204750,8061.34,5.311,101411207,113074431,18688.88727,8.053
190497,und,255151,1,367531,14470.29,5.565,157099890,160198542,26477.53754,8.205
190498,die,326305,1,409915,16139.02,5.613,170961824,204236386,33756.09110,8.310


How much do the items in MultiPic overlap with tokens in SUBTLEX-DE?

In [9]:
# lowercase a string and remove umlauts
def remove_umlauts(string):
    """ Removes umlauts and ß and lowercases strings.
    Input:
        string: A string.
    Output:
        new_string: Same string in lowercase and without umlauts.
    """
    umlauts = {'ä':'ae','ö':'oe','ü':'ue','ß':'ss'}
    new_string = string.lower()
    for umlaut in umlauts:
        new_string = new_string.replace(umlaut, umlauts[umlaut])
    return new_string

In [10]:
# get values
## MultiPic
name1_vals = set(multipic_df['NAME1'].values)
name2_vals = set(multipic_df['NAME2'].values)
name2_vals.remove(np.nan)
# combine entries from NAME1 and NAME2 columns
multipic_vals = name1_vals.union(name2_vals)
## SUBTLEX-DE
subtlex_vals = set(subtlex_df['Word'])
# make tokens comparable to MultiPic 
# by lowercasing + removing umlauts and ß
# create dict to map original to cleaned token
subtlex_vals_dict = dict()
for token in subtlex_vals:
    cleaned = remove_umlauts(token)
    # create new cleaned dict key if it doesn't exist yet
    if not cleaned in subtlex_vals_dict.keys():
        subtlex_vals_dict[cleaned] = []
    # map original to cleaned token
    subtlex_vals_dict[cleaned].append(token)

# create a set variable for easier calling
subtlex_vals_cleaned = set(subtlex_vals_dict.keys())

In [11]:
len(subtlex_vals_cleaned)

190073

In [12]:
len(subtlex_vals)

190500

In [13]:
# check the overlap between the databases
present = multipic_vals.intersection(subtlex_vals_cleaned)
not_present = multipic_vals.difference(present)
print('There is an overlap of',len(present), 
        'tokens between the MultiPic version 1 (Duñabeitia et al. 2018) and the SUBTLEX-DE database;',
        len(not_present),'items can\'t be inferred from the SUBTLEX-DE database.')

There is an overlap of 702 tokens between the MultiPic version 1 (Duñabeitia et al. 2018) and the SUBTLEX-DE database; 12 items can't be inferred from the SUBTLEX-DE database.


What are the 12 items that the SUBTLEX-DE database doesn't account for?

In [14]:
not_present

{'billardloch',
 'chameleon',
 'chinesische mauer',
 'cupcake',
 'dartpfeil',
 'lastkraftwagen',
 'malerrolle',
 'ritterruestung',
 't-shirt',
 'torrero',
 'treppenstufe',
 'u-boot'}

Check whether different versions or writing styles are present in SUBTLEX:

In [15]:
for database in [subtlex_vals_cleaned, multipic_vals]:
    if database == subtlex_vals_cleaned:
        print('SUBTLEX')
    else:
        print('\nMultiPic')
    for token in ['dart', 'billard', 'lkw', 'ritter', 'uboot', 'tshirt', 'stufe']:
        if token in database:
            print(token)

SUBTLEX
dart
billard
lkw
ritter
uboot
tshirt
stufe

MultiPic
ritter


In [16]:
multipic_df[multipic_df['NAME1']=='ritter']

,ITEM,NAME1,NAME2,H_INDEX,PERCENTAGE_MODAL_NAME,PERCENTAGE_VALID_RESPONSES,NUMBER_DIFFERENT_RESPONSES,PERCENTAGE_UNKNOWN_RESPONSES,PERCENTAGE_IDIOSYNCRATIC_RESPONSES,VISUAL_COMPLEXITY
431,432,ritter,NaN,2.34787,42.857143,91,9,1,8,3.56


In [17]:
multipic_df[multipic_df['NAME1']=='ritterruestung']

,ITEM,NAME1,NAME2,H_INDEX,PERCENTAGE_MODAL_NAME,PERCENTAGE_VALID_RESPONSES,NUMBER_DIFFERENT_RESPONSES,PERCENTAGE_UNKNOWN_RESPONSES,PERCENTAGE_IDIOSYNCRATIC_RESPONSES,VISUAL_COMPLEXITY
175,176,ritterruestung,NaN,1.550643,43.75,96,3,0,4,3.06


We could probably take the frequencies of "uboot" and "tshirt" from the SUBTLEX-DE corpus still; since "ritter" and "ritterrüstung" seem to be their own items in MultiPic we shouldn't substitute for that, and "lkw" might also be too far away from "lastkraftwagen" for us to just take it.

In [18]:
# load relevant columns from SUBTLEX
test = ['tshirt','Tshirt','TShirt','uboot','Uboot','UBoot']
for word in test:
    if word in subtlex_vals:
        print(word)

TShirt
UBoot


Now we know for sure how 'tshirt' and 'uboot' are written in the SUBTLEX corpus.

# Check out "inconsistency" in SUBTLEX-DE

As we saw earlier in this notebook, there is quite a difference between the number of unique items in SUBTLEX-DE when comparing cleaned (lowercased + without umlauts) to non-cleaned entries (190073 to 190500). So what words are there two different writings of?

In [19]:
from collections import Counter
subtlex_cleaned_list = []
for token in subtlex_vals:
    subtlex_cleaned_list.append(remove_umlauts(token))

c = Counter(subtlex_cleaned_list)

In [20]:
# check 5 most common duplicates
c.most_common(5)

[('hoeflich', 2), ('krueger', 2), ('hoelle', 2), ('bae', 2), ('kueche', 2)]

Are any of the duplicate items ones we need for adding frequencies to the MultiPic items?

In [21]:
# turn Counter object into a dictionary
counts = dict(c)
# filter to get list of duplicate entries
duplicates = [key for key, value in counts.items() if value > 1]
duplicates = set(duplicates)

# check overlap between MultiPic items and duplicates
problematic_cases = multipic_vals.intersection(duplicates)
print(f'There are {len(problematic_cases)} problematic cases.')

There are 15 problematic cases.


In [22]:
# create dict of probelamtic cases with info from token mapping dict
problematic_cases_dict = {key:value for key, value in subtlex_vals_dict.items() if key in problematic_cases}
problematic_cases_dict

{'kueche': ['Küche', 'Kueche'],
 'wueste': ['Wüste', 'Wueste'],
 'schluessel': ['Schluessel', 'Schlüssel'],
 'baer': ['Bär', 'Baer'],
 'gefaengnis': ['Gefängnis', 'Gefaengnis'],
 'maedchen': ['Maedchen', 'Mädchen'],
 'kaese': ['Kaese', 'Käse'],
 'bruecke': ['Brücke', 'bruecke'],
 'koenig': ['Koenig', 'König'],
 'schaedel': ['Schädel', 'Schaedel'],
 'tuer': ['Tuer', 'Tür'],
 'schaefer': ['Schaefer', 'Schäfer'],
 'jaeger': ['Jäger', 'Jaeger'],
 'fruehstueck': ['Fruehstueck', 'Frühstück'],
 'ruecken': ['ruecken', 'Rücken']}

Are there different frequencies for the different writings in SUBTLEX-DE?

In [23]:
# create empty dataframe for collection of problematic cases
columns = ['Word','WFfreqcount','spell-check OK (1/0)','CUMfreqcountm','SUBTLEX','lgSUBTLEX','Google00','Google00cum','Google00pm','lgGoogle00']
problematic_cases_df = pd.DataFrame(columns=columns)

# get set of problematic original entries
problematic_tokens = []
for token in problematic_cases_dict.values():
    problematic_tokens += token
problematic_tokens = set(problematic_tokens)

# add problematic entry to dataframe
for i in subtlex_df.index:
    if subtlex_df.loc[i,'Word'] in problematic_tokens:
        problematic_cases_df.loc[len(problematic_cases_df.index)] = subtlex_df.loc[i]

In [24]:
problematic_cases_df

,Word,WFfreqcount,spell-check OK (1/0),CUMfreqcountm,SUBTLEX,lgSUBTLEX,Google00,Google00cum,Google00pm,lgGoogle00
0,Wueste,2,0,NaN,0.08,0.477,29,58,0.00959,1.771
1,Fruehstueck,2,0,NaN,0.08,0.477,63,63,0.01041,1.806
2,Kaese,7,0,NaN,0.28,0.903,119,119,0.01967,2.079
3,Kueche,4,0,NaN,0.24,0.845,121,121,0.02,2.086
4,Gefaengnis,4,0,NaN,0.16,0.699,128,128,0.02116,2.111
5,bruecke,4,0,NaN,0.16,0.699,55,135,0.02231,2.134
6,ruecken,2,0,NaN,0.08,0.477,35,160,0.02644,2.207
7,Schaedel,1,0,NaN,0.04,0.301,170,185,0.03058,2.27
8,Schluessel,3,0,NaN,0.12,0.602,274,430,0.07107,2.634
9,Maedchen,17,0,NaN,0.79,1.322,523,526,0.08694,2.722


The duplicates are cleanly divided into ones that are spelled correctly (that also have a way higher frequency) and ones that are spelled incorrectly. Naturally, we will choose the entries with the correct spelling.

### Checking out difference between correct and incorrect spellings

In [25]:
correct_spelling = subtlex_df[subtlex_df['spell-check OK (1/0)']==1]
correct_vals = set(correct_spelling['Word'])
correct_vals_cleaned = set()
for token in correct_vals:
    correct_vals_cleaned.add(remove_umlauts(token))
print(len(correct_vals))
print(len(correct_vals_cleaned))

correct_present = multipic_vals.intersection(correct_vals_cleaned)
correct_not_present = multipic_vals.difference(correct_present)

print('Only considering correct spellings, there is an overlap of',len(correct_present), 
        'tokens between the MultiPic version 1 (Duñabeitia et al. 2018) and the SUBTLEX-DE database;',
        len(correct_not_present),'items can\'t be inferred from the SUBTLEX-DE database.')

125486
125486
Only considering correct spellings, there is an overlap of 684 tokens between the MultiPic version 1 (Duñabeitia et al. 2018) and the SUBTLEX-DE database; 30 items can't be inferred from the SUBTLEX-DE database.


In [26]:
correct_not_present

{'billardloch',
 'burger',
 'chameleon',
 'chinesische mauer',
 'cupcake',
 'dartpfeil',
 'dartscheibe',
 'donut',
 'gladiator',
 'kiwi',
 'koala',
 'laetzchen',
 'lastkraftwagen',
 'limousine',
 'maeppchen',
 'malerrolle',
 'muffin',
 'playstation',
 'pylone',
 'rasenmaeher',
 'rasierer',
 'ritterruestung',
 'roller',
 'skateboard',
 't-shirt',
 'tacker',
 'torrero',
 'treppenstufe',
 'tukan',
 'u-boot'}

In [27]:
subtlex_df[subtlex_df['Word']=='Roller']

,Word,WFfreqcount,spell-check OK (1/0),CUMfreqcount,SUBTLEX,lgSUBTLEX,Google00,Google00cum,Google00pm,lgGoogle00
163515,Roller,75,0,75,2.95,1.881,8857,9776,1.61577,3.99


It seems that some "real" words just aren't accounted for in the spell-check database. It might be best to ignore the spell-check column except for cases with more than one orthographic variant.

# Combine information from MultiPic and SUBTLEX-DE

**NOTE: The following code is also accessible as a reproducible script. See `merge_multipic_subtlex.py` in the `src` directory.**


Take the following information:

From MultiPic:
- ITEM
- PICTURE
- NAME1
- H_INDEX
- PERCENTAGE_MODAL_NAME
- VISUAL_COMPLEXITY

From SUBTLEX-DE:
- SUBTLEX
- lgSUBTLEX
- Google00pm
- lgGoogle00

In [28]:
# first, take relevant info from MultiPic
combined_df = pd.read_csv(multipic_path,sep=';', decimal=',', usecols=['ITEM','PICTURE','NAME1','H_INDEX','PERCENTAGE_MODAL_NAME','VISUAL_COMPLEXITY'])

# prepare relevant SUBTLEX columns
combined_df = combined_df.reindex(columns=combined_df.columns.tolist()+['SUBTLEX','lgSUBTLEX','Google00pm','lgGoogle00'])

# fill frequency cells
for token in multipic_vals:
    # remove any '-' (concerns u-boot and t-shirt)
    token = token.replace('-','')
    # check if token is present in SUBTLEX-DE
    if token in subtlex_vals_cleaned:
        # get correct row in SUBTLEX-DE
        index = 0
        # if there is more than one orthographic variant in SUBTLEX-DE
        if len(subtlex_vals_dict[token]) > 1:
            # check if entry 2 has correct spelling
            variant = subtlex_vals_dict[token][1]
            row = subtlex_df[(subtlex_df['Word'] == variant) & (subtlex_df['spell-check OK (1/0)'] == 1)]
            if len(row) > 0:
                # if yes, take note of index
                index = row.index[0]
        # else take row index from first variant
        variant = subtlex_vals_dict[token][0]
        index = subtlex_df[subtlex_df['Word'] == variant].index[0]
        
        # get frequency information
        subt = subtlex_df.loc[index, 'SUBTLEX']
        lgsubt = subtlex_df.loc[index, 'lgSUBTLEX']
        google = subtlex_df.loc[index, 'Google00pm']
        lggoogle =subtlex_df.loc[index, 'lgGoogle00']

        # account for the fact that some names occur more than once
        # add frequency information to combined_df
        indices = combined_df[combined_df['NAME1'] == token].index
        for i in range(len(indices)):
            combined_df.loc[indices[i], 'SUBTLEX'] = subt
            combined_df.loc[indices[i], 'lgSUBTLEX'] = lgsubt
            combined_df.loc[indices[i], 'Google00pm'] = google
            combined_df.loc[indices[i], 'lgGoogle00'] = lggoogle

combined_df

,ITEM,PICTURE,NAME1,H_INDEX,PERCENTAGE_MODAL_NAME,VISUAL_COMPLEXITY,SUBTLEX,lgSUBTLEX,Google00pm,lgGoogle00
0,1,PICTURE_1,maus,0.000000,100.000000,3.02,10.91,2.444,11.17323,4.830
1,2,PICTURE_2,reifen,1.163235,69.696970,2.67,15.75,2.603,7.10619,4.633
2,3,PICTURE_3,treppenstufe,1.426663,46.464646,1.89,NaN,NaN,NaN,NaN
3,4,PICTURE_4,altar,0.968778,77.319588,3.40,7.56,2.286,5.93453,4.555
4,5,PICTURE_5,schulter,0.862355,82.978723,2.01,22.52,2.758,12.59959,4.882
...,...,...,...,...,...,...,...,...,...,...
745,746,PICTURE_746,kegel,0.516599,90.721649,1.69,0.91,1.380,1.80303,4.038
746,747,PICTURE_747,badewanne,0.468996,90.000000,2.57,7.91,2.305,1.58503,3.982
747,748,PICTURE_748,muschel,1.102087,73.195876,3.25,1.61,1.623,0.91945,3.745
748,749,PICTURE_749,muelleimer,1.678743,50.000000,3.32,3.35,1.934,0.70227,3.628
